#  Описание проекта
__Банки — Анализ оттока клиентов__
Необходимо проанализировать клиентов регионального банка «Метанпром» и выделить сегменты клиентов, которые склонны уходить из банка.

- Провести исследовательский анализ данных,
- Выделить портреты клиентов, которые склонны уходить из банка,
- Сформулировать и проверите статистические гипотезы.

<a id='presentation'></a>
## Презентация

https://drive.google.com/file/d/10FNCRIT3BGlPj-xcA_O_vxaJVD5GzMlh/view?usp=share_link

# Описание датасетов

Датасет содержит данные о клиентах банка «Метанпром». Банк располагается в Ярославле и областных городах: Ростов Великий и Рыбинск.

Колонки:

- `userid` — идентификатор пользователя,
- `score` — баллы кредитного скоринга,
- `City` — город,
- `Gender` — пол,
- `Age` — возраст,
- `Equity` — приблизительная оценка собственности клиента,
- `Balance` — баланс на счёте,
- `Products` — количество продуктов, которыми пользуется клиент,
- `CreditCard` — есть ли кредитная карта,
- `last_activity` — активность клиента 1(активный)/0(неактивный),
- `est_salary` — заработная плата клиента,
- `Churn` — ушёл или нет.

# Содержание

1. [Открытие данных и общая информация](#start)


2. [Предобработка данных](#preprocessing)
 
    2.1 [Приведение заголовков столбцов к "змеиному" регистру](#registre_columns)
    
    2.2 [Изменение типов данных](#type)
    
    2.3 [Обработка пропущенных значений](#null)
    
    2.4 [Обработка дубликатов](#duplicates)
       
    2.5 [Обработка аномальных значений](#anomalies)
    
    2.6 [Создание столбца с категорией города (цифра соответствует определённому городу)](#banks)
    
   
3. [Анализ данных](#analyse)

    3.1 [Определить зависимость баллов кредитного скоринга и вероятности ухода](#score)
    
    3.2 [Определить зависимость вероятности ухода от города](#city)
    
    3.3 [Какого пола клиенты чаще уходят](#gender)
    
    3.4 [Определить зависимость вероятности ухода от возраста](#age)
    
    3.5 [Определить зависимость вероятности ухода от оценки собственности клиента](#equity)
    
    3.6 [Определить зависимость вероятности ухода от баланса на счёте](#balance)
    
    3.7 [Определить зависимость вероятности ухода от количества продуктов, которыми пользуется клиент](#products)
    
    3.8 [Определить зависимость вероятности ухода от наличия кредитной карты](#credit_card)
    
    3.9 [Определить зависимость вероятности ухода от активности](#loyalty)
    
    3.10 [Определить зависимость вероятности ухода от заработной платы](#salary)
    
    3.11 [Произвести анализ вероятности ухода в зависимости сразу от нескольких критериев](#cross_analysis)
    
    3.12 [Составить портрет клиента, склонного уходить](#portraits)
    
    3.13 [Выводы](#conclusions)
 

4. [Проверка 3-х гипотез](#stat_test)

    4.1 [Формулирование гипотез](#hypotesis)
    
    4.2 [Тестирование первой гипотезы](#H1)
    
    4.3 [Тестирование второй гипотезы](#H2)
    
    4.4 [Тестирование третьей гипотезы](#H3)
    
    4.5 [Выводы](#conclusions1)
    
    
5. [Презентация](#presentation)

<a id='start'></a>
## Открытие данных и оющая информация

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as st
import plotly.figure_factory as ff
import plotly.express as px
from plotly import graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
path = "https://drive.google.com/uc?export=download&id=1-U61mhTz_N1ARjy2XSAZ7IlQqGjeqP0F"
df = pd.read_csv(path)
df.head(5)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.hist(figsize=(15,20));

В датасете 10000 строк и 12 столбцов. Только в столбце balance есть пропуски. Каких-то аномальных данных из "описания" пока не видно.

<a id='preprocessing'></a>
## Предобработка данных

<a id='registre_columns'></a>
### Приведение заголовков столбцов к "змеиному" регистру

In [ ]:
df.columns = df.columns.str.lower()
df.columns

<a id='type'></a>
### Изменение типов данных

Имеет смысл у столбца score изменить тип данных  с float64 на uint16 (целые числа в диапазоне от 0 по 65535) т.к. это целые значения от 0 до 1000. 

Также у столбцов age, equity, products, credit_card, last_activity, churn изменить тип с int64  на uint8 (целые числа в диапазоне от 0 по 255) для ускорения расчётов и экономии оперативной памяти.

In [ ]:
df[['age','equity', 'products', 'credit_card', 'last_activity', 'churn']] = \
df[['age','equity', 'products', 'credit_card', 'last_activity', 'churn']] .astype('uint8')
df['score'] = df['score'].astype('uint16')

<a id='null'></a>
### Обработка пропущенных значений

In [ ]:
#проверим данные на наличие пропусков
print(df.isna().sum())

In [ ]:
fig = px.histogram(df, x="balance", color="churn", marginal="rug",
                   hover_data=df.columns, title='Распределение данных столбца balance')
fig.update_layout(xaxis_title='Баланс счёта клиента')
fig.show()

Распределение данных скошено вправо в сторону высоких значений баланса, поэтому заполнять пропуски средним значением будет ошибкой. Заполним пропуски медианным значением балланса, медиану посчитаем у уходящих и остающихся клиентов отдельно, предполагая, что есть разница в балансе, что подтверждает график выше.

<a id='dublicates'></a>
### Обработка дубликатов

Проверим уникальные названия городов, если есть повторяющиеся с разными регистрами, приведём их к одному

In [ ]:
df['city'].unique()

С городами всё в порядке. Проверим тоже самое с в столбуе gender

In [ ]:
df['gender'].unique()

Тоже всё в порядке

In [ ]:
#проверим количесво дубликатов
df.duplicated().sum()

In [ ]:
# удалим дубликат и убедимся, что удалили его
df = df[df.duplicated()!=True]
df.duplicated().sum()

<a id='anomalies'></a>
### Обработка аномальных значений

In [ ]:
df.describe()

Аномальные значения могут встретиться в столбцах age, balance  и est_salary. Построим по ним диаграммы размаха.

In [ ]:
plt.title("Диаграмма размаха  данных столбца 'age'")  
ax=sns.boxplot(x = df['age'])
ax.set(xlabel='Возраст клиентов')  
plt.show()

Старше 77 лет уже не совсем типичные клиенты банка.

In [ ]:
plt.title("Диаграмма размаха  данных столбца 'balance'")  
ax=sns.boxplot(x = df['balance'])
ax.set(xlabel='Баланс счёта') 
plt.show()

Данные сильно смещены, очень много выбросов, которые могут искажать картину. Чтобы немного выровнять имеет смысл удалить пользователей с балансом выше 20 млн.

In [ ]:
plt.title("Диаграмма размаха  данных столбца 'est_salary'")  
ax=sns.boxplot(x = df['est_salary'])
ax.set(xlabel='Предполагаемый ежемесячный доход') 
plt.show()

 Данные тоже смещены. Все данные "за усами" исключить нельзя, т.к. мы лишаемся большого количества данных. Попробуем взять границу в 600 тыс.


Посчитаем какое количество данных мы удалим, применяя критерии выше.

In [ ]:
print(round(len(df[(df['balance'] > 20000000) | (df['est_salary'] > 600000) | (df['age'] > 77)])/len(df['userid'])*100,2), '%')

 __2.62% это приемлемое количество данных для удаления (<5). Удаляем__


In [ ]:
index= df[(df['balance'] > 20000000) | (df['est_salary']> 600000) |(df['age'] > 77)].index.to_list()

In [ ]:
df1 = df.drop(index = index)
len(df1)

In [ ]:
df =df1.copy()

<a id='banks'></a>
### Создание дополнительных столбцов с категорией города в котором находится банк  и с гендерной принадлежностью клиента

In [ ]:
df2 = pd.get_dummies (df, columns=['city', 'gender'], drop_first= False)

In [ ]:
df2.head()

In [ ]:
df = df2.copy()

<a id='analyse'></a>
## Анализ данных

In [ ]:
cm = df.corr() 
fig, ax = plt.subplots(figsize = (14, 14))


sns.heatmap(cm, annot = True, square=True, cmap="plasma")

plt.title('Матрица корреляций')
plt.show()

__Согласно матрице наибольшая связь с оттоком клиентов наблюдается у следующих параметров:__

- оценка собственности клиента (equity)

- баланс счёта (balance)

- количество используемых продуктов банка (products)


<a id='score'></a>
### Определить зависимость баллов кредитного скоринга и вероятности ухода

Для определения зависимости баллов кредитного скоринга (кредитный рейтинг) и вероятности уходя сделаем следующие действия:

- посчитаем коэфициэнт корреляции Пирсона, 

- построим распределение данных в зависимости от вероятности ухода

- сравним медианные или средние значения исследуемого параметра в зависимости от вероятности ухода.

In [ ]:
df['score'].corr(df['churn']).round(2)

In [ ]:
fig = px.histogram(df, x="score", color="churn", marginal="rug",
                   hover_data=df.columns, title='Распределение кредитного рейтинга в зависимости от оттока клиентов')
fig.update_layout(xaxis_title='Кредитный рейтинг')
fig.show()

In [ ]:
df.pivot_table(index = 'churn', values = 'score', aggfunc = 'median').reset_index()

Кредитный скоринг остающихся клиентов более равномерно распределён от 0 до 1000,  кредитный скоринг уходящих клиентов начинается от 700 и максимально сосредоточен в области 800-930. 

У уходящих клиентов медианный кредитный скоринг 866, у остающихся 847.

<a id='city'></a>
### Определить зависимость вероятности ухода от города

In [ ]:
specs = [[{'type':'domain'}, {'type':'domain'}, {'type': 'domain'}]]
colors = ['rgb(33, 75, 99)', 'rgb(79, 129, 102)']
fig = make_subplots(rows=1, cols=3, specs=specs)

# Define pie charts
fig.add_trace(go.Pie(labels=df['churn'], values=df['city_Ярославль'], name='Ярославль',
                     marker_colors=colors, scalegroup='one', title='Ярославль'), 1, 1)
fig.add_trace(go.Pie(labels=df['churn'], values=df['city_Рыбинск'],name='Рыбинск',
                     marker_colors=colors, scalegroup='one', title= 'Рыбинск'), 1, 2)
fig.add_trace(go.Pie(labels=df['churn'], values=df['city_Ростов'], name='Ростов',
                     marker_colors=colors, scalegroup='one', title= 'Ростов'), 1, 3)

fig.update(layout_title_text='Соотношение уходящих и не уходящих клиентов по городам',
           layout_showlegend=True)

fig = go.Figure(fig)
fig.show()

Больше всего клиентов уходит в Ярославле ( как в количественном выражении, так и в процентном) 19,2%  - 1094 чел.

<a id='gender'></a>
### Какого пола клиенты чаще уходят

In [ ]:
df_gender1 =  df.pivot_table(index = 'gender_Ж', values = 'churn', aggfunc = 'sum').reset_index()
df_gender1

Отток мужчин в 2 раза больше, чем женщин. 

Посмотрим сколько мужчин и женщин вообще обслуживается в банке 

In [ ]:
df_gender = df.groupby(['gender_Ж'])[['gender_Ж']].count().rename(columns={'gender_Ж': 'count'}).reset_index()
df_gender

In [ ]:
specs = [[{'type':'domain'}, {'type':'domain'}]]
colors = ['rgb(129, 180, 179)', 'rgb(124, 103, 37)']
fig = make_subplots(rows=1, cols=2, specs=specs)

fig.add_trace(go.Pie(labels=df_gender1['gender_Ж'], values=df_gender1['churn'], name='Ушедшие клиенты',
                     marker_colors=colors, title='Ушедшие клиенты. 0- мужчины, 1- женщины', textinfo='label+percent'), 1, 1)
fig.add_trace(go.Pie(labels=df_gender['gender_Ж'], values=df_gender['count'],name='0- мужчины, 1- женщины',
                     marker_colors=colors,  title= 'Все клиенты. 0- мужчины, 1- женщины', textinfo='label+percent'), 1, 2)

fig.update(layout_title_text='Зависимость оттока клиентов от пола',
           layout_showlegend=True)

fig = go.Figure(fig)
fig.show()

__В общей выборке клиенты по половому признаку делятся практически пополам. В категории ушедших клиентов мужчин почти в 2 раза больше чем женщинЖ (64,8% мужчин и 35,2% женщин)__

<a id='age'></a>
### Определить зависимость вероятности ухода от возраста

Для определения зависимости баллов кредитного скоринга (кредитный рейтинг) и вероятности уходя сделаем следующие действия:

- посчитаем коэфициэнт корреляции Пирсона, 

- построим распределение данных в зависимости от вероятности ухода

- сравним медианные или средние значения исследуемого параметра в зависимости от вероятности ухода.

In [ ]:
df['age'].corr(df['churn']).round(2)

In [ ]:
fig = px.histogram(df, x="age", color="churn", marginal="rug",
                   hover_data=df.columns, title='Распределение возраста в зависимости от оттока клиентов')
fig.update_layout(xaxis_title='Возраст')
fig.show()

Распределение не симметрично, поэтому для сравнения "среднего" возраста ушедших и не ушедших клиентов будем использовать медиану

In [ ]:
df.pivot_table(index = 'churn', values = 'age', aggfunc = 'median')

__Сильной разницы в возрасте между отточными  и не отточными клиентами не наблюдается. Самый частый возраст оттока с 30 до 40 лет__

<a id='equity'></a>
### Определить зависимость вероятности ухода от оценки собственности клиента

In [ ]:
df['equity'].corr(df['churn']).round(2)

In [ ]:
df_no_churn = df.loc[df['churn']==0].groupby(['equity'])[['churn']].count()\
.rename(columns={'churn': 'count'}).reset_index().sort_values(by='count', ascending= False)
df_no_churn

In [ ]:
df_churn = df.loc[df['churn']==1].groupby(['equity'])[['churn']].count()\
.rename(columns={'churn': 'count'}).reset_index().sort_values(by='count', ascending= False)
df_churn

In [ ]:
specs = [[{'type':'domain'}, {'type':'domain'}]]
colors = ['rgb(129, 180, 179)', 'rgb(124, 103, 37)']
fig = make_subplots(rows=1, cols=2, specs=specs)

fig.add_trace(go.Pie(labels=df_churn['equity'], values=df_churn['count'], name='Ушедшие клиенты',
                     marker_colors=colors, title='Ушедшие клиенты', textinfo='label+percent'), 1, 1)
fig.add_trace(go.Pie(labels=df_no_churn['equity'], values=df_no_churn['count'],name='Оставшиеся клиенты',
                     marker_colors=colors,  title= 'Оставшиеся', textinfo='label+percent'), 1, 2)

fig.update(layout_title_text='Распледеление клиентов в зависимости от оценки собственности среди ушедших и оставшихся',
           layout_showlegend=True)

fig = go.Figure(fig)
fig.show()

Среди ушедших клиентов наибольшее количество 31% имели оценку собственности 5,на втором месте 26% -4. Среди оставшихся немного иное распределение: наибольшее количество имеют оценку 0 - 30%, на втором месте также оценка 4 - 17,2%  

Посмотрим соотношение оставшихся и ушедших клиентов в зависимости от оценки собственности.

In [ ]:
df_group=df.groupby(['equity', 'churn'])[['userid']].count().rename(columns={'userid': 'count'}).reset_index()
#добавим проценты
df_group['ratio'] = (df_group['count']/df_group.groupby('equity')['count'].transform('sum')).round(2)*100
df_group

In [ ]:
fig = px.bar(df_group, x='equity', y='ratio', color = 'churn',\
             title='Зависимость оттока клиентов от оценки собственности клиента')
fig.update_layout(xaxis_title='Оценка собственности',
                   yaxis_title='Доля клиентов, %')
fig.show()

Наибольшая доля ушедших клиенто приходится на оценку собственности - 9. Заметна тенденция увеличения доли ушедших клиентов у увеличением оценки собственности (оценка 8 выбивается из немного из этой тенденции)

__На мой взгляд зависимоть от оценки собственности есть. С увеличением оценки увеличивается отток.Об этом говорит последний график, коэфициент Пирсона тоже.  Конечно клиентов с оценкой собственности больше 7 не так много, чтобы делать однозначный вывод, но по крайней мере до 7  - тенденция явно просматривается.__

<a id='balance'></a>
### Определить зависимость вероятности ухода от баланса на счёте

In [ ]:
df['balance'].corr(df['churn']).round(2)

In [ ]:
fig = px.histogram(df, x="balance", color="churn", marginal="rug",
                   hover_data=df.columns, title='Распределение баланса счёта в зависимости от оттока клиентов')
fig.update_layout(xaxis_title='Баланс, руб')
fig.show()

In [ ]:
df_balance = df.pivot_table(index = 'churn', values = 'balance', aggfunc = 'median').reset_index()
df_balance

In [ ]:
fig = px.bar(df_balance, x='churn', y='balance', \
             title='Зависимость оттока от  баланса')
fig.update_layout(xaxis_title='1: клиент ушёл, 0: клиент остаётся',
                   yaxis_title='Баланс, руб')
fig.show()

__В среднем у уходящих клиентов баланс счета выше, чем у остающихся. Наибольшее количество ушедших клиентов имеет баланс от 70 до 900 тыс__

<a id='products'></a>
### Определить зависимость вероятности ухода от количества продуктов, которыми пользуется клиент

In [ ]:
df['products'].corr(df['churn']).round(2)

In [ ]:
df_products=df.groupby(['products', 'churn'])[['userid']].count().rename(columns={'userid': 'count'}).reset_index()
#добавим проценты
df_products['ratio'] = (df_products['count']/df_products.groupby('products')['count'].transform('sum')).round(2)*100
df_products

In [ ]:
fig = px.bar(df_products, x='products', y='ratio', color = 'churn',\
             title='Зависимость оттока клиентов от количества используемых продуктов')
fig.update_layout(xaxis_title='Количество продуктов',
                   yaxis_title='Доля клиентов, %')
fig.show()

Доля уходящих клиентов увеличивается при увеличении количества продуктов от 1 до 4, но если клиент начинает пользоваться 5-ю продутками, то вероятность что он лстанется уведичивается. Наибольшая доля оттока у клиентов пользующихся 4-мя продуктами.

In [ ]:
df_products.loc[df_products['churn']==1]

In [ ]:
fig = px.bar(df_products.loc[df_products['churn']==1], x='products', y='count', \
             title='Зависимость оттока клиентов от количества используемых продуктов')
fig.update_layout(xaxis_title='Количество продуктов',
                   yaxis_title='Количество ушедших клиентов')
fig.show()

Если смотреть по абсолютному количеству, максимальный отток наблюдается у пользователей с 2-мя продуктами.Но этих пользователей в принципе больше. Думаю, что всё же корректнее смотреть на долю, а не на абсолютное значение.

<a id='credit_card'></a>
### Определить зависимость вероятности ухода от наличия кредитной карты

In [ ]:
df.pivot_table(index = 'credit_card', values = 'churn', aggfunc = 'sum').reset_index()

Среди ушедших клиентов чуть больше половины имели кредитные карты.

In [ ]:
df_credit = df.groupby(['credit_card'])[['credit_card']].count().rename(columns={'credit_card': 'count'}).reset_index()
df_credit

In [ ]:
specs = [[{'type':'domain'}, {'type':'domain'}]]
colors = ['rgb(129, 180, 179)', 'rgb(124, 103, 37)']
fig = make_subplots(rows=1, cols=2, specs=specs)

fig.add_trace(go.Pie(labels=df['credit_card'], values=df['churn'], name='Ушедшие клиенты',
                     marker_colors=colors, title='Ушедшие клиенты', textinfo='label+percent'), 1, 1)
fig.add_trace(go.Pie(labels=df_credit['credit_card'], values=df_credit['count'],name='Все клиенты',
                     marker_colors=colors,  title= 'Все клиенты', textinfo='label+percent'), 1, 2)

fig.update(layout_title_text='Зависимость оттока клиентов от наличия кредитной карты. 1- есть карта, 0- нет карты',
           layout_showlegend=True)

fig = go.Figure(fig)
fig.show()

Доля клиентов с кредитными картами среди ушедших клиентов меньше, чем среди общего количества клиентов на 12 %.

Склоняюсь, что зависимость есть, хоть и небольшая. Отрицательный коэффициент корреляции тоже говорит об обратной связи. Логически если ты платишь кредит, то ты просто не можешь уйти из банка, пока не погасишь его.

<a id='loyalty'></a>
### Определить зависимость вероятности ухода от активности

In [ ]:
df['last_activity'].corr(df['churn']).round(2)

In [ ]:
df_activity = df.pivot_table(index = 'last_activity', values = 'churn', aggfunc = 'sum').reset_index()
df_activity

In [ ]:
fig = px.bar(df_activity, x='last_activity', y='churn',\
             title='Зависимость оттока клиентов от активности')
fig.update_layout(xaxis_title='1: активный клиент, 0: неактивный',
                   yaxis_title='Количество ушедших клиентов, руб')
fig
fig.show()

Как это ни странно, среди ушедших пользователей активных было больше чем не автивных больше чем в 2 раза.

Посмотрим доли доли ушедших пользователей среди общего количества с разбивкой по активности. 

In [ ]:
df_activity1=df.groupby(['last_activity', 'churn'])[['userid']].count().rename(columns={'userid': 'count'}).reset_index()
#добавим проценты
df_activity1['ratio'] = (df_activity1['count']/df_activity1.groupby('last_activity')['count'].transform('sum')).round(2)*100
df_activity1

In [ ]:
fig = px.bar(df_activity1, x='last_activity', y='ratio', color = 'churn',\
             title='Зависимость оттока клиентов от активности пользователей. churn: 0 - оставшиеся, 1- ушедшие клиенты')
fig.update_layout(xaxis_title='Активность клиента: 1-активный, 0 - неактивный',
                   yaxis_title='Доля клиентов, %')
fig.show()

Доля ушедших клиентов среди активных пользователей больше примерно на 20%

<a id='salary'></a>
### Определить зависимость вероятности ухода от заработной платы

In [ ]:
fig = px.histogram(df, x="est_salary", color="churn", marginal="rug",
                   hover_data=df.columns, title='Распределение предполагаемого дохода в зависимости от оттока клиентов')
fig.update_layout(xaxis_title='Предполагаемый доход, руб')
fig.show()

In [ ]:
df_salary = df.pivot_table(index = 'churn', values = 'est_salary', aggfunc = 'median').reset_index()
df_salary

In [ ]:
fig = px.bar(df_salary, x='churn', y='est_salary',\
             title='Зависимость оттока клиентов от дохода')
fig.update_layout(xaxis_title='Отток: 1-ушёл, 0 - остался',
                   yaxis_title='Медианный доход, руб')
fig
fig.show()

Распределение не симметрично, для сравнения посчитали медианы. Есть небольшое различие. У ушедших клиентов медианный доход немного больше, чем у оставшихся.

<a id='cross_analysis'></a>
### Произвести анализ вероятности ухода в зависимости сразу от нескольких критериев

 Выделим несколько сегментов, в которых как мы думаем высока доля ушедших клиентов и посмотрим какой процентоттока получится при сочетании этих сегментов.
Посчитаем 3 варианта:

1. Мужчины  с оценкой собственности 7 и выше и балансом больше 400 тыс

2. Возраст 30-40, пользуются 4-мя продуктами, кредитный рейтинг больше 800

3. Зарплата выше 70тыс,  активные пользователи, пользуются 4-мя продуктами


In [ ]:
df_seg1 = df.loc[(df['gender_М']== 1)  & (df['balance'] > 400000) & (df['equity'] >= 7)]
df_seg1 = df_seg1.groupby(['churn'])[['userid']].count().rename(columns={'userid': 'count'}).reset_index()
df_seg1['ratio'] = (df_seg1['count']/(df_seg1['count'].sum())).round(2)*100
df_seg1

In [ ]:
df_seg2 = df.loc[(df['age'] > 30 ) & (df['age'] < 40) & (df['score'] > 800) &(df['products'] ==4)]
df_seg2 = df_seg2.groupby(['churn'])[['userid']].count().rename(columns={'userid': 'count'}).reset_index()
df_seg2['ratio'] = (df_seg2['count']/(df_seg2['count'].sum())).round(2)*100
df_seg2

In [ ]:
df_seg3 = df.loc[(df['est_salary'] > 70000 ) & (df['last_activity'] == 1) &(df['products'] ==4)]
df_seg3 = df_seg3.groupby(['churn'])[['userid']].count().rename(columns={'userid': 'count'}).reset_index()
df_seg3['ratio'] = (df_seg3['count']/(df_seg3['count'].sum())).round(2)*100
df_seg3

In [ ]:
specs = [[{'type':'domain'}, {'type':'domain'}, {'type': 'domain'}]]
colors = ['rgb(33, 75, 99)', 'rgb(79, 129, 102)']
fig = make_subplots(rows=1, cols=3, specs=specs)

# Define pie charts
fig.add_trace(go.Pie(labels=df_seg1['churn'], values=df_seg1['count'], name='Первый вариант сегментации',
                     marker_colors=colors, title='Вариант 1'), 1, 1)
fig.add_trace(go.Pie(labels=df_seg2['churn'], values=df_seg2['count'], name='Второй вариант сегментации',
                     marker_colors=colors, title='Вариант 2'), 1, 2)
fig.add_trace(go.Pie(labels=df_seg3['churn'], values=df_seg3['count'], name='Третий вариант сегментации',
                     marker_colors=colors, title='Вариант 3'), 1, 3)

fig.update(layout_title_text='Анализ вероятности оттока в зависимости от разных критериев (сегментаций)',
           layout_showlegend=True)

fig = go.Figure(fig)
fig.show()

 Наибольший процент оттока в третьем варианте сегментации: зарплата выше 70 тыс, пользующихся 4-мя продуктами и имеющие оценку собственности выше 7.
 
 Я пробовала разные сочетания параметров. В итоге наибольшее влияние оказывает то, что клиент пользуется 4-мя продуктами банка.

<a id='portraits'></a>
### Составить портрет клиента, склонного уходить

__Согласно проведённым исследованиям, чаще уходят из банка:__

1. Мужчины в возрасте 30-40 лет

2. С зарплатой больше 70 тыс. руб

3. Пользующиеся 4-мя продуктами

4. Имеющие высокий кредитный скор от 800

5. Активный пользователь банка

6. С высокой оценкой собственности (выше 6)

Одним словом уходят успешные, активные клиенты.

<a id='conclusions'></a>
### Выводы

Наибольшее влияние на отток клиентов оказывают следующие параметры:

- пользование 4-мя продуктами: 
доля уходящих клиентов увеличивается при увеличении количества продуктов от 1 до 4, но если клиент начинает пользоваться 5-ю продутками, то вероятность что он уйдёт уменьшится. Наибольшая доля оттока у клиентов пользующихся 4-мя продуктами ~63%.

- активность: доля ушедших клиентов среди активных пользователей больше примерно на 20%

- гендерная принадлежность к мужскому полу: ~65% отточных клиентов мужчины

- кредитный рейтинг выше 800: кредитный скоринг остающихся клиентов более равномерно распределён от 0 до 1000, кредитный скоринг уходящих клиентов начинается от 700 и максимально сосредоточен в области 800-930.
У уходящих клиентов медианный кредитный скоринг 866, у остающихся 847

- оценка собвственности: с увеличением оценки увеличивается отток. Наибольшая доля ушедших клиентов приходится на оценку собственности - 9-54%

- имеют медианный баланс выше чем неотточные клиенты: медианный баланс отточных клиентов ~770тыс. руб., остающихся ~472 тыс. руб., 

- имеют медианный доход выше чем неотточные клиенты: медианный доход отточных клиентов ~124тыс. руб., остающихся ~116 тыс. руб., 

Параметры, которые меньше  влияют на отток клиентов:

- Город. По всем 3 городам примерно одинаковый % оттока клиентов (от 16 до 19%%)

- Наличие кредитной карты: доля клиентов с кредитными картами среди ушедших клиентов меньше, чем среди общего количества клиентов на 12 

__Рекомендации:__

1. Провести более детальный анализ влияния пользования продуктами банка на отток клиентов. По логике, чем большим количеством продуктов пользуется клиент, тем он более лоялен к банку. У нас получается наоборот.

2. Сформировать выгодные предложения и продукты, ориентированные на портрет отточных клиентов - успешных, активных людей. 

 <a id='stat_test'></a>
## Проверка 3-х гипотез

<a id='hypotesis'></a>
### Формулирование гипотез

__Гипотеза 1. Средние доходы пользователей, которые ушли и остались, различаются__

__Гипотеза 2. Активность пользователей, которые ушли  и остались, различаются__

__Гипотеза 3. Количество продуктов, которыми пользовались ушедшие и оставшиеся клиенты различаются__

<a id='H1'></a>
### Тестирование первой гипотезы


H_0: Доход клиентов, которые ушли = доход клиентов, которые остались

H_1: Доход клиентов, которые ушли ≠ доход клиентов, которые остались

alpha = 0.05

In [ ]:
fig = px.histogram(df, x="est_salary", color="churn", marginal="rug",
                   hover_data=df.columns, title='Распределение предполагаемого дохода в зависимости от оттока клиентов')
fig.update_layout(xaxis_title='Предполагаемый доход, руб')
fig.show()

Чтобы мы могли применять для расчётов t-критерий Стьюдента необходимо, чтобы наши данные были распределены __нормально__, чтобы __выборки были не зависимы друг от друга__ и __дисперсии рассматриваемых генеральных совокупностей были__.

Наши данные соответствуют всем условиям, кроме последнего. В равенстве дисперсии мы не можем быть уверены, и поскольку и по количеству данных выборки сильно отличаются, в расчёте мы укажем параметр equal_var = False  

In [ ]:
churn_1 =  df[df['churn'] == 1]['est_salary'] 
churn_0 = df[df['churn'] == 0]['est_salary'] 
results = st.ttest_ind(churn_1, churn_0, equal_var = False) # results = вызов метода для проверки гипотезы

alpha = 0.05 

print(results.pvalue) 
if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Не получилось отвергнуть нулевую гипотезу')

__Средние доходы оставшихся и ушедших клиентов не равны__

<a id='H2'></a>
### Тестирование второй гипотезы

H_0: Активность клиентов, которые ушли = активности клиентов, которые остались

H_1: Активность клиентов, которые ушли ≠ активности клиентов, которые остались

    alpha = 0.05

In [ ]:
fig = px.histogram(df, x="last_activity", color="churn", marginal="rug",
                   hover_data=df.columns, title='Активности в зависимости от оттока клиентов')
fig.update_layout(xaxis_title='Активность: 0- не активный, 1- активный', yaxis_title = "Количество пользователей")
fig.show()

Мы имеем дело с непараметрическими данными. Поэтому будем использовать тест Манна-Уитни

In [ ]:
churn_1 =  df[df['churn'] == 1]['last_activity'] 
churn_0 = df[df['churn'] == 0]['last_activity'] 
results = st.mannwhitneyu(churn_1, churn_0)

print('p-значение: ', results.pvalue)

alpha = 0.05

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print(
        'Не получилось отвергнуть нулевую гипотезу'
    ) 

__Активность ушедших и оставшихся клиентов не равны__

<a id='H3'></a>
### Тестирование третьей гипотезы

H_0: Количество продуктов клиентов, которые ушли = количество продуктов клиентов, которые остались

H_1: Количество продуктов клиентов, которые ушли ≠ количество продуктов клиентов, которые остались

    alpha = 0.05

In [ ]:
fig = px.histogram(df, x="products", color="churn", marginal="rug",
                   hover_data=df.columns, title='Распределение количества продуктов в зависимости от оттока клиентов')
fig.update_layout(xaxis_title='Количество продуктов', yaxis_title = "Количество пользователей")
fig.show()

Мы имеем дело с непараметрическими данными. Поэтому будем использовать тест Манна-Уитни

In [ ]:
churn_1 =  df[df['churn'] == 1]['products'] 
churn_0 = df[df['churn'] == 0]['products'] 
results = st.mannwhitneyu(churn_1, churn_0)

print('p-значение: ', results.pvalue)

alpha = 0.05

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу: разница статистически значима')
else:
    print(
        'Не получилось отвергнуть нулевую гипотезу, вывод о различии сделать нельзя'
    ) 

__Количество продуктов , которыми пользовались ушедшие клиенты отличается от количества продуктов, которыми пользуются оставшиеся клиенты__

<a id='conclusion1'></a>
### Выводы

__Проверка гипотез подтвердила наши более ранние выводы о различном поведении клиентов которые ушли и которые остались__

<a id='presentation'></a>
## Презентация

https://drive.google.com/file/d/10FNCRIT3BGlPj-xcA_O_vxaJVD5GzMlh/view?usp=share_link